In [34]:
!pip install --upgrade gensim pyLDAvis mlflow
!pip install protobuf==3.19.0
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install graphviz
!pip install --upgrade nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 8.4 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/12.8 MB 23.5 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 25.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import re
import os
import sys
import pickle
import gensim
import spacy
import pyLDAvis
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim import corpora
from datetime import datetime
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models as gensimvis
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.tree import export_graphviz
import graphviz

tqdm.pandas() 


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import mlflow
import mlflow.sklearn

C:\Users\misty.hickman\AppData\Roaming\Python\Python312\site-packages\google\protobuf\internal\well_known_types.py:92: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  _EPOCH_DATETIME = datetime.utcfromtimestamp(0)
C:\Users\misty.hickman\AppData\Roaming\Python\Python312\site-packages\mlflow\gateway\config.py:64: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("togetherai_api_key", pre=True)
C:\Users\misty.hickman\AppData\Roaming\Python\Python312\site-packages\mlflow\gateway\config.py:78: PydanticDeprecatedSince20: P

In [2]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.matutils import Sparse2Corpus


# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\misty.hickman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\misty.hickman\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\misty.hickman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\misty.hickman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
today = datetime.today().strftime("%Y-%m-%d-%I:%M:%S%p")
run_dir = f"/model_artifacts/run_{today}"
print(f"The mode artifacts will be saved in {run_dir}")

The mode artifacts will be saved in /model_artifacts/run_2025-01-18-07:50:14AM


In [10]:
train_df = pd.read_csv('research_review_data.csv')
#display the requests table
train_df.columns

Index(['reviews', 'response_reviews', 'start_date', 'end_date'], dtype='object')

In [11]:
# Function to preprocess text
def preprocess_text(text):
    """
    Preprocess text by tokenizing, lowercasing, removing stopwords, and lemmatizing.
    """
    tokens = nltk.word_tokenize(text.lower())  # Tokenize and lowercase
    lemmatized_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words
    ]
    return " ".join(lemmatized_tokens)

In [12]:
train_df.head()

,reviews,response_reviews,start_date,end_date
0,The focus is on presenting a balanced perspect...,Focus on connecting your findings back to the ...,12/7/2024 10:33,12/28/2024 13:33
1,I began researching the topic of biology by fo...,"Your initial review is detailed, but consider ...",11/4/2024 13:33,11/10/2024 22:33
2,This research aims to provide a comprehensive ...,Ensure your conclusions align with the evidenc...,10/23/2024 8:33,11/12/2024 8:33
3,Time management was essential for completing t...,Consider narrowing your focus to make the argu...,12/1/2024 0:33,12/5/2024 6:33
4,One challenge was finding sufficient resources...,Excellent use of resources; keep up the thorou...,11/13/2024 6:33,11/26/2024 11:33


In [15]:
#creating new processed columns
train_df['processed_reviews'] = train_df['reviews'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else [])
train_df['processed_response_reviews'] = train_df['response_reviews'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else [])

#concatenate memo and memo response for topic modeling  
train_df['combined_text'] = (
    train_df['processed_reviews'].fillna("") + " " + train_df['processed_response_reviews'].fillna("")
)


print(train_df.columns)  # Check all column names
print(train_df[['combined_text']].head())  # Check the contents of combined_text

Index(['reviews', 'response_reviews', 'start_date', 'end_date',
       'processed_reviews', 'processed_response_reviews', 'combined_text'],
      dtype='object')
                                       combined_text
0  focus presenting balanced perspective addressi...
1  began researching topic biology focusing write...
2  research aim provide comprehensive understandi...
3  time management essential completing task rela...
4  one challenge finding sufficient resource art ...


In [16]:
# Create a TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limit vocabulary size to most important terms
    stop_words='english',
    max_df=0.95,  # Ignore terms in more than 95% of documents
    min_df=2      # Ignore terms in fewer than 2 documents
)
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df['combined_text'])

In [17]:
# Map the TF-IDF matrix to Gensim-compatible corpus
gensim_corpus = Sparse2Corpus(tfidf_matrix, documents_columns=False)
gensim_dictionary = Dictionary([tfidf_vectorizer.get_feature_names_out()])

# Output basic stats
print(f"Number of unique tokens in dictionary: {len(gensim_dictionary)}")
print(f"TF-IDF Corpus created with {len(gensim_corpus)} documents.")

Number of unique tokens in dictionary: 136
TF-IDF Corpus created with 100 documents.


In [18]:
# Train the LDA model using Gensim
from gensim.models.ldamodel import LdaModel

# Define the number of topics
num_topics = 10

# Train the LDA model
lda_model = LdaModel(
    corpus=gensim_corpus,
    num_topics=num_topics,
    id2word=gensim_dictionary,
    passes=10,
    random_state=42
)

# Print topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")


Topic 0: 0.059*"historygreat" + 0.040*"sciencestrong" + 0.038*"systematicallyensure" + 0.038*"historyexcellent" + 0.027*"scienceexcellent" + 0.009*"psychology" + 0.008*"computer" + 0.007*"summarize" + 0.007*"event" + 0.007*"focus"
Topic 1: 0.074*"event" + 0.074*"summarize" + 0.050*"economicsgreat" + 0.041*"historyconsider" + 0.030*"biologymight" + 0.007*"biology" + 0.007*"key" + 0.007*"focus" + 0.007*"angle" + 0.007*"point"
Topic 2: 0.057*"systematicallyexcellent" + 0.038*"systematicallymight" + 0.007*"consider" + 0.007*"ensure" + 0.007*"key" + 0.007*"history" + 0.007*"point" + 0.007*"use" + 0.007*"analysis" + 0.007*"art"
Topic 3: 0.096*"physic" + 0.087*"philosophy" + 0.049*"supported" + 0.049*"building" + 0.023*"biologyinitial" + 0.016*"databaseensure" + 0.014*"physicfocus" + 0.013*"study" + 0.013*"key" + 0.011*"include"
Topic 4: 0.010*"systematicallyexcellent" + 0.010*"event" + 0.010*"summarize" + 0.008*"biology" + 0.008*"psychology" + 0.008*"strong" + 0.008*"computer" + 0.008*"resea